# **Mount Google Drive and Install Dependencies**
This section mounts your Google Drive to Colab so you can save files to it, and installs the necessary dependencies like `streamlit` for creating the chatbot interface and `localtunnel` for exposing the app to the web.


In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Install Streamlit and Localtunnel
!pip install streamlit
!npm install -g localtunnel

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 62.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 85.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.4 MB/s eta 0:00:00
⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏
added 22 packages in 4s
⠏
⠏3 packages are looking for funding
⠏  run `npm fund` for details
⠏npm notice
npm notice New major version of npm available! 10.8.2 -> 11.0.0
npm notice Changelog: https://github.com/npm/cli/releases/tag/v11.0.0
npm notice To update run: npm install -g npm@11.0.0
npm notice
⠏

# **Write the Chatbot Application Code**
This section contains the full implementation of the chatbot. The code includes:
- Loading intents data from a JSON file.
- Preprocessing data using TF-IDF and encoding tags.
- Training a Random Forest classifier.
- Setting up a Streamlit interface with features such as chat, chat history, model evaluation, and an about page.


In [2]:
# Write the chatbot app to a file
app_code = """
import os
import json
import random
import nltk
import streamlit as st
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from datetime import datetime

# Download necessary NLTK datasets
nltk.download('punkt')

# Function to load data from the intents file
def load_data(file_path):
    with open(file_path, 'r') as file:
        intents = json.load(file)
    patterns, tags, responses = [], [], {}
    for intent in intents:
        for pattern in intent['patterns']:
            patterns.append(pattern)
            tags.append(intent['tag'])
        responses[intent['tag']] = intent['responses']
    return patterns, tags, responses

# Function to preprocess data
def preprocess_data(patterns, tags):
    vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
    label_encoder = LabelEncoder()
    x = vectorizer.fit_transform(patterns)
    y = label_encoder.fit_transform(tags)
    return x, y, vectorizer, label_encoder

# Chatbot prediction function
def chatbot(input_text, clf, vectorizer, label_encoder, responses):
    input_vector = vectorizer.transform([input_text])
    predicted_tag_index = clf.predict(input_vector)[0]
    predicted_tag = label_encoder.inverse_transform([predicted_tag_index])[0]
    return random.choice(responses.get(predicted_tag, ["I'm not sure how to respond to that."]))

# Streamlit Chat Interface
def display_chat():
    st.title("Enhanced Intent-Based Chatbot")

    # Sidebar menu
    menu = ["Home", "Chat History", "Model Evaluation", "About"]
    choice = st.sidebar.selectbox("Menu", menu)

    # Initialize chat history
    if 'chat_history' not in st.session_state:
        st.session_state['chat_history'] = []

    # Load data and train model
    patterns, tags, responses = load_data('/content/drive/My Drive/intents.json')
    x, y, vectorizer, label_encoder = preprocess_data(patterns, tags)
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
    clf = RandomForestClassifier(n_estimators=200, random_state=42)
    clf.fit(X_train, y_train)

    if choice == "Home":
        st.write("Welcome! Ask the chatbot a question.")
        input_container = st.empty()

        with input_container.container():
            user_input = st.text_input("You:", key="user_input", placeholder="Type your message here...")
            send_button = st.button("Send")

        if user_input or send_button:
            if user_input:
                response = chatbot(user_input, clf, vectorizer, label_encoder, responses)
                st.session_state['chat_history'].append({
                    "user": user_input,
                    "chatbot": response,
                    "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                })
                st.write(f"You: {user_input}")
                st.write(f"Chatbot: {response}")

    elif choice == "Chat History":
        st.subheader("Chat History")
        if st.session_state['chat_history']:
            for chat in st.session_state['chat_history']:
                st.write(f"**You:** {chat['user']}")
                st.write(f"**Chatbot:** {chat['chatbot']}")
                st.write(f"**Timestamp:** {chat['timestamp']}")
                st.markdown("---")
        else:
            st.write("No chat history available.")

    elif choice == "Model Evaluation":
        st.subheader("Model Evaluation")
        model_accuracy = accuracy_score(y_test, clf.predict(X_test))
        classification_rep = classification_report(y_test, clf.predict(X_test))
        st.write(f"Model Accuracy: {model_accuracy * 100:.2f}%")
        st.write("### Classification Report")
        st.text(classification_rep)

    elif choice == "About":
        st.write("This project is a chatbot built using NLP and Streamlit.")

if __name__ == '__main__':
    display_chat()
"""

# **Write to Drive and Verify File Creation**
This section writes the app file to drive and ensures that the chatbot application file was successfully written to the specified location in your Google Drive.


In [3]:
# Save the app code to a Python file
with open('/content/drive/My Drive/app.py', 'w') as f:
    f.write(app_code)
# Verify the file is saved
print("Chatbot app written to /content/drive/My Drive/app.py")

Chatbot app written to /content/drive/My Drive/app.py


# **Set Up Localtunnel Password**
This section uses a secure password for Localtunnel to authenticate and establish a connection for exposing the Streamlit app to the public. Ensure that you have the correct credentials to proceed.


In [4]:
# Authenticate with Localtunnel using a password
!curl https://loca.lt/mytunnelpassword

34.133.209.251

# **Run Streamlit Application**
This section starts the Streamlit server to run the chatbot application. The app's logs are redirected to a file in the Colab environment for debugging purposes.

In [5]:
# Start the Streamlit app
!streamlit run /content/drive/My\ Drive/app.py &>/content/logs.txt &

# **Set Up Localtunnel for Public Access**
This section uses Localtunnel to expose the Streamlit app to the public, allowing you to access it via a unique URL.

In [6]:
# Set up localtunnel to expose the app
!npx localtunnel --port 8501

⠙⠹⠸⠼⠴⠦⠧⠇your url is: https://metal-tools-retire.loca.lt
^C


# **Final Steps**
Once all the code sections have been executed, follow these steps to run the Streamlit app:

1. **Obtain the Tunnel Password**: After running the Localtunnel password command, you will receive a password for the tunnel. Make sure to copy this password.

2. **Access the Tunnel URL**: Localtunnel will provide a unique URL that you can use to access the Streamlit app. Open this URL in your browser.

3. **Enter the Tunnel Password**: Paste the tunnel password when prompted.

4. **Streamlit App Running**: The Streamlit app should now be live and accessible through the provided URL.

**Caution**:
- If you encounter a **Bad Gateway** error, try running the Localtunnel setup again or reload the tunnel URL. The app will run fine once the tunnel is properly established.
